In [1]:
import os
from osgeo import gdal, osr

In [2]:
def calculate_overlap(tile1_path, tile2_path):
    # Open the raster datasets
    tile1_dataset = gdal.Open(tile1_path)
    tile2_dataset = gdal.Open(tile2_path)

    # Get the geotransform information (bounding box) of the tiles
    geotransform1 = tile1_dataset.GetGeoTransform()
    geotransform2 = tile2_dataset.GetGeoTransform()

    # Calculate overlapping area
    x_min1, y_max1 = geotransform1[0], geotransform1[3]
    x_max1, y_min1 = x_min1 + geotransform1[1] * tile1_dataset.RasterXSize, y_max1 + geotransform1[5] * tile1_dataset.RasterYSize

    x_min2, y_max2 = geotransform2[0], geotransform2[3]
    x_max2, y_min2 = x_min2 + geotransform2[1] * tile2_dataset.RasterXSize, y_max2 + geotransform2[5] * tile2_dataset.RasterYSize

    x_overlap = max(0, min(x_max1, x_max2) - max(x_min1, x_min2))
    y_overlap = max(0, min(y_max1, y_max2) - max(y_min1, y_min2))

    return x_overlap * y_overlap

def is_overlapping(tile_path, independent_tiles):
    for independent_tile_path in independent_tiles:
        if calculate_overlap(tile_path, independent_tile_path) > 0:
            return True
    return False

def find_non_overlapping_tiles(tile_directory):
    all_tile_paths = [os.path.join(tile_directory, file) for file in os.listdir(tile_directory) if file.endswith(".tif")]
    independent_tile_paths = [all_tile_paths[0]]

    for tile_path in all_tile_paths[1:]:
        if not is_overlapping(tile_path, independent_tile_paths):
            independent_tile_paths.append(tile_path)

    return independent_tile_paths

In [3]:
# Example usage
tile_directory = "C:\\Users\\rowanconverse\\Desktop\\PowerAnalysis\\BdA\\18bw"
independent_tile_paths = find_non_overlapping_tiles(tile_directory)

print("Independent tile paths:")
for tile_path in independent_tile_paths:
    print(tile_path)

Independent tile paths:
C:\Users\rowanconverse\Desktop\PowerAnalysis\BdA\18bw\BDA_18bw_20181107_00378_Projected.tif
C:\Users\rowanconverse\Desktop\PowerAnalysis\BdA\18bw\BDA_18bw_20181107_00384_Projected.tif
C:\Users\rowanconverse\Desktop\PowerAnalysis\BdA\18bw\BDA_18bw_20181107_00396_Projected.tif
C:\Users\rowanconverse\Desktop\PowerAnalysis\BdA\18bw\BDA_18bw_20181107_00402_Projected.tif
C:\Users\rowanconverse\Desktop\PowerAnalysis\BdA\18bw\BDA_18bw_20181107_00407_Projected.tif
C:\Users\rowanconverse\Desktop\PowerAnalysis\BdA\18bw\BDA_18bw_20181107_00413_Projected.tif
C:\Users\rowanconverse\Desktop\PowerAnalysis\BdA\18bw\BDA_18bw_20181107_00450_Projected.tif
C:\Users\rowanconverse\Desktop\PowerAnalysis\BdA\18bw\BDA_18bw_20181107_00454_Projected.tif
C:\Users\rowanconverse\Desktop\PowerAnalysis\BdA\18bw\BDA_18bw_20181107_00485_Projected.tif
C:\Users\rowanconverse\Desktop\PowerAnalysis\BdA\18bw\BDA_18bw_20181107_00490_Projected.tif
C:\Users\rowanconverse\Desktop\PowerAnalysis\BdA\18bw\BD

In [9]:
#Derive detection radius for projected tile

import numpy as np
import rasterio
from rasterio.transform import from_origin

def create_circular_clip(input_path, output_path):
    with rasterio.open(input_path) as src:
        data = src.read()
        num_bands = src.count
        
        # Create a mask for null values (0, 0, 0) across all bands
        null_mask = np.all(data == 0, axis=0)
        
        center_x = src.width / 2
        center_y = src.height / 2
        
        min_distance_x = min(center_x, src.width - center_x)
        min_distance_y = min(center_y, src.height - center_y)
        radius_pixels = int(min(min_distance_x, min_distance_y))
        
        y_coords, x_coords = np.ogrid[:src.height, :src.width]
        dist_from_center = np.sqrt((x_coords - center_x)**2 + (y_coords - center_y)**2)
        circular_mask = dist_from_center <= radius_pixels
        
        combined_mask = ~null_mask & circular_mask
        
        circular_clip = np.zeros_like(data, dtype=src.dtypes[0])
        for band in range(num_bands):
            circular_clip[band] = data[band] * combined_mask
        
        clipped_transform = from_origin(
            src.transform.xoff, src.transform.yoff,
            src.transform.a, src.transform.a
        )
        
        with rasterio.open(output_path, 'w', driver='GTiff', width=src.width, height=src.height,
                           count=num_bands, dtype=src.dtypes[0], crs=src.crs,
                           transform=clipped_transform) as dst:
            for band in range(num_bands):
                dst.write(circular_clip[band], band + 1)

# Specify input and output file paths
input_path = 'C:\\Users\\rowanconverse\\Desktop\\PowerAnalysis\\BdA\\18bw\\BDA_18bw_20181107_00378_Projected.tif'
output_path = 'C:\\Users\\rowanconverse\\Desktop\\PowerAnalysis\\BdA\\test_clip6.tif'

create_circular_clip(input_path, output_path)


In [10]:
#Iterate the detection radius clips over a directory

import os

# Input and output directories
input_dir = 'C:\\Users\\rowanconverse\\Desktop\\PowerAnalysis\\BdA\\18bw'
output_dir = 'C:\\Users\\rowanconverse\\Desktop\\PowerAnalysis\\BdA\\18bw_radius'

# Iterate over files in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith('.tif'):  # Assuming your tiles are GeoTIFFs
        input_path = os.path.join(input_dir, filename)
        output_filename = f"{os.path.splitext(filename)[0]}_clip.tif"
        output_path = os.path.join(output_dir, output_filename)
        
        create_circular_clip(input_path, output_path)

print("Clipping complete.")


Clipping complete.
